In [1]:
# Импортируем все необходимые библиотеки и задаем сид для рандомизатора
import pandas as pd
import numpy as np
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

np.random.seed(1)

Следующая ячейка предназначена для сбора данных.
Рекомендуемые датасеты уже внесены в paths
Можно добавлять новые, но не стоит/не рекомендуется использовать video_games.csv, english.csv и подобные

In [2]:
# Считываем данные
paths = [
    r"data\handed.csv",
    r"data\generalBG.csv",
    r"data\logicBG.csv",
    r"data\educationalBG.csv",
]

datasets = []

for path in paths:
    reviews = pd.read_csv(path, index_col=0)
    datasets.append(reviews)

# Объединение в один датасет
data = pd.concat(datasets).reset_index(drop=True)
data

,review,assessment
0,"Игра, в которой победить может даже злодей. От...",True
1,Обалденная игра для компании. Помогает разнооб...,True
2,"Весёлая настольная игра для компании, которая ...",True
3,Общие впечатления Игра огонь даже для взрослог...,True
4,Случайно наткнулся на эту игрушку в детском ми...,True
...,...,...
1776,Мелкие игровые карточки.\nНо мы решили их заме...,True
1777,"Немного не понравилось, что фон домино яркий, ...",True
1778,"Классная игра, играем в троем с ребенком 4 лет...",True
1779,"Пластик качественный,приятные цвета,интересное...",True


In [3]:
# Количество позитивных (True) и отрицательных отзывов (False)
data['assessment'].value_counts()

assessment
True     1567
False     214
Name: count, dtype: int64

In [4]:
# Кодируем классы отзывов
enc = LabelEncoder()
label = enc.fit_transform(data["assessment"])
label

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [5]:
# Предобработка текста

processed = data["review"].str.lower()
processed


nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))


processed = processed.apply(
    lambda x: " ".join(term for term in x.split() if term not in stop_words)
)

ps = nltk.PorterStemmer()
processed = processed.apply(lambda x: " ".join(ps.stem(term) for term in x.split()))

tokenizer = RegexpTokenizer(r"\w+")
processed = processed.apply(lambda x: " ".join(tokenizer.tokenize(x)))
processed

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       игра которой победить злодей отличное карточно...
1       обалденная игра компании помогает разнообразит...
2       весёлая настольная игра компании которая раскр...
3       общие впечатления игра огонь взрослого питание...
4       случайно наткнулся игрушку детском мире учитыв...
                              ...                        
1776    мелкие игровые карточки решили заметить картам...
1777    немного понравилось фон домино яркий счет карт...
1778    классная игра играем троем ребенком 4 лет прои...
1779    пластик качественный приятные цвета интересное...
1780                    очень понравился квест рекомендую
Name: review, Length: 1781, dtype: object

In [6]:
# Создаем словарь с наиболее часто встречаемыми словами

nltk.download("punkt_tab")
all_words = []

for text in processed:
    words = word_tokenize(text)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print("Number of words: {}".format(len(all_words)))
print("Most common words: {}".format(all_words.most_common(15)))
word_features = [x[0] for x in all_words.most_common(2000)]

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Number of words: 7955
Most common words: [('игра', 829), ('очень', 631), ('лет', 299), ('игры', 271), ('это', 237), ('играть', 221), ('5', 217), ('карточки', 186), ('3', 179), ('качество', 168), ('2', 166), ('хорошая', 166), ('интересная', 157), ('просто', 156), ('играем', 155)]


In [7]:
# Функция для нахождения фич в тексте
def find_features(text):
    words = word_tokenize(text)
    features = {}
    for word in word_features:
        features[word] = word in words

    return features

In [8]:
# Получаем данные для тренировки (75%) и тестов (25%)
reviews = list(zip(processed, label))

np.random.shuffle(reviews)

feature_set = [(find_features(text), label) for (text, label) in reviews]

training, test = train_test_split(feature_set, test_size=0.25, random_state=1)

print(len(training))
print(len(test))

1335
446


In [9]:
# Обучение байесовского классификатора
clf = MultinomialNB()
mulnb_model = SklearnClassifier(clf).train(training)

In [10]:
# Получаем метрики точности
text_features, labels = zip(*test)
prediction = mulnb_model.classify_many(text_features)

print(classification_report(labels, prediction))


pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

              precision    recall  f1-score   support

           0       0.65      0.67      0.66        45
           1       0.96      0.96      0.96       401

    accuracy                           0.93       446
   macro avg       0.81      0.81      0.81       446
weighted avg       0.93      0.93      0.93       446



predicted         
                 negative positive
actual negative        30       15
       positive        16      385

In [11]:
# Обучение модели логистической регрессии
clf = LogisticRegression(max_iter=100)
logreg_model = SklearnClassifier(clf).train(training)

In [12]:
# Получаем метрики точности для второй модели
text_features, labels = zip(*test)
prediction = logreg_model.classify_many(text_features)

print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

              precision    recall  f1-score   support

           0       0.78      0.62      0.69        45
           1       0.96      0.98      0.97       401

    accuracy                           0.94       446
   macro avg       0.87      0.80      0.83       446
weighted avg       0.94      0.94      0.94       446



predicted         
                 negative positive
actual negative        28       17
       positive         8      393

In [ ]:
# Сохранение первой модели со словарем фич и функцие нахождения их в тексте
path = r"models\naive_bayes_classifier.pickle"

with open(path, "wb") as classifier_file:
    data_for_save = {
        'model': mulnb_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

In [ ]:
# Сохранение второй модели со словарем фич и функцие нахождения их в тексте
path = r"models\logistic_regression_classifier.pickle"

with open(path, "wb") as classifier_file:
    data_for_save = {
        "model": logreg_model,
        "features": word_features,
        "function": find_features,
    }
    pickle.dump(data_for_save, classifier_file)